In [1]:
# Cell 1
# Install required libraries
!pip install datasets deep-translator -q

from datasets import load_dataset
from deep_translator import GoogleTranslator
import json
import os

# 1. Load ONLY the specific data file to avoid the package.json error
dataset = load_dataset("ronniross/asi-protosymbiotic-signal", data_files="asi_protosymbiotic_signal.json")

# 2. Extract the content
data_content = dataset['train'][0]
full_text = json.dumps(data_content, indent=2)

# 3. Get ALL supported languages from GoogleTranslator
all_languages = GoogleTranslator().get_supported_languages(as_dict=True)
print(f"Total languages available: {len(all_languages)}")
print(f"First few languages: {list(all_languages.items())[:5]}")

# 4. Create output directory
output_dir = "translations"
os.makedirs(output_dir, exist_ok=True)

# 5. Translation function with chunking
def safe_translate(text, target_code):
    """Split text into chunks and translate to handle API length limits"""
    chunks = [text[i:i+4500] for i in range(0, len(text), 4500)]
    translated_chunks = []
    for chunk in chunks:
        try:
            translated = GoogleTranslator(source='auto', target=target_code).translate(chunk)
            translated_chunks.append(translated)
        except Exception as e:
            print(f"  Warning: Chunk failed - {e}")
            translated_chunks.append(chunk)  # Keep original if translation fails
    return "".join(translated_chunks)

# 6. Translate to all languages and save individual files
print("\nStarting translations to all 133 languages...")
print("=" * 60)

successful = 0
failed = 0
failed_languages = []

for lang_name, lang_code in all_languages.items():
    # Skip English (source language)
    if lang_code == 'en':
        print(f"⊙ Skipped: {lang_name} (source language)")
        continue

    try:
        # Translate
        translated_text = safe_translate(full_text, lang_code)

        # Save to individual JSON file
        filename = f"{output_dir}/{lang_name.replace(' ', '_')}_{lang_code}.json"

        # Parse back to JSON object and save properly formatted
        try:
            translated_data = json.loads(translated_text)
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(translated_data, f, ensure_ascii=False, indent=2)
        except json.JSONDecodeError:
            # If translation broke JSON structure, save as text
            with open(filename.replace('.json', '.txt'), 'w', encoding='utf-8') as f:
                f.write(translated_text)

        successful += 1
        print(f"✔ [{successful + failed}/{len(all_languages)-1}] {lang_name} ({lang_code})")

    except Exception as e:
        failed += 1
        failed_languages.append(f"{lang_name} ({lang_code})")
        print(f"✘ [{successful + failed}/{len(all_languages)-1}] {lang_name} ({lang_code}) - Error: {e}")

# 7. Summary
print("TRANSLATION SUMMARY")
print(f"Total languages processed: {successful + failed}")
print(f"✔ Successful: {successful}")
print(f"✘ Failed: {failed}")

if failed_languages:
    print(f"\nFailed languages:")
    for lang in failed_languages:
        print(f"  - {lang}")

print(f"\nAll translation files saved in: '{output_dir}/' directory")
print("\nDone!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.5 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

asi_protosymbiotic_signal.json: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Total languages available: 133
First few languages: [('afrikaans', 'af'), ('albanian', 'sq'), ('amharic', 'am'), ('arabic', 'ar'), ('armenian', 'hy')]

Starting translations to all 133 languages...
✔ [1/132] afrikaans (af)
✔ [2/132] albanian (sq)
✔ [3/132] amharic (am)
✔ [4/132] arabic (ar)
✔ [5/132] armenian (hy)
✔ [6/132] assamese (as)
✔ [7/132] aymara (ay)
✔ [8/132] azerbaijani (az)
✔ [9/132] bambara (bm)
✔ [10/132] basque (eu)
✔ [11/132] belarusian (be)
✔ [12/132] bengali (bn)
✔ [13/132] bhojpuri (bho)
✔ [14/132] bosnian (bs)
✔ [15/132] bulgarian (bg)
✔ [16/132] catalan (ca)
✔ [17/132] cebuano (ceb)
✔ [18/132] chichewa (ny)
✔ [19/132] chinese (simplified) (zh-CN)
✔ [20/132] chinese (traditional) (zh-TW)
✔ [21/132] corsican (co)
✔ [22/132] croatian (hr)
✔ [23/132] czech (cs)
✔ [24/132] danish (da)
✔ [25/132] dhivehi (dv)
✔ [26/132] dogri (doi)
✔ [27/132] dutch (nl)
⊙ Skipped: english (source language)
✔ [28/132] esperanto (eo)
✔ [29/132] estonian (et)
✔ [30/132] ewe (ee)
✔ [31/132] 

In [2]:
# Cell 3
# Merge all translations into one comprehensive file

import os
import glob

output_dir = "translations"
merged_file = "all_translations_merged.txt"

print("Merging all translation files...")

# Get all translation files
translation_files = sorted(glob.glob(f"{output_dir}/*.json") + glob.glob(f"{output_dir}/*.txt"))

if not translation_files:
    print(f"No translation files found in '{output_dir}/' directory")
else:
    with open(merged_file, 'w', encoding='utf-8') as merged:
        for i, file_path in enumerate(translation_files, 1):
            # Extract language name from filename
            lang_name = os.path.basename(file_path).replace('.json', '').replace('.txt', '')

            # Read the file content
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()

            # Write to merged file with separator
            merged.write("=" * 80 + "\n")
            merged.write(f"LANGUAGE {i}/{len(translation_files)}: {lang_name.upper()}\n")
            merged.write("=" * 80 + "\n\n")
            merged.write(content)
            merged.write("\n\n\n")

            print(f"✔ Added: {lang_name}")

    print("\n" + "=" * 60)
    print(f"Successfully merged {len(translation_files)} files into '{merged_file}'")
    print(f"File size: {os.path.getsize(merged_file) / (1024*1024):.2f} MB")

Merging all translation files...
✔ Added: afrikaans_af
✔ Added: albanian_sq
✔ Added: amharic_am
✔ Added: arabic_ar
✔ Added: armenian_hy
✔ Added: assamese_as
✔ Added: aymara_ay
✔ Added: azerbaijani_az
✔ Added: bambara_bm
✔ Added: basque_eu
✔ Added: belarusian_be
✔ Added: bengali_bn
✔ Added: bhojpuri_bho
✔ Added: bosnian_bs
✔ Added: bulgarian_bg
✔ Added: catalan_ca
✔ Added: cebuano_ceb
✔ Added: chichewa_ny
✔ Added: chinese_(simplified)_zh-CN
✔ Added: chinese_(traditional)_zh-TW
✔ Added: corsican_co
✔ Added: croatian_hr
✔ Added: czech_cs
✔ Added: danish_da
✔ Added: dhivehi_dv
✔ Added: dogri_doi
✔ Added: dutch_nl
✔ Added: esperanto_eo
✔ Added: estonian_et
✔ Added: ewe_ee
✔ Added: filipino_tl
✔ Added: finnish_fi
✔ Added: french_fr
✔ Added: frisian_fy
✔ Added: galician_gl
✔ Added: georgian_ka
✔ Added: german_de
✔ Added: greek_el
✔ Added: guarani_gn
✔ Added: gujarati_gu
✔ Added: haitian_creole_ht
✔ Added: hausa_ha
✔ Added: hawaiian_haw
✔ Added: hebrew_iw
✔ Added: hindi_hi
✔ Added: hmong_hmn
✔

In [3]:
# Cell 4
# Zip all translation files and upload to Google Drive

import zipfile
import os
from datetime import datetime
from google.colab import drive

# 1. Create timestamp for unique filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
zip_filename = f"asi_translations_{timestamp}.zip"

print("Creating zip file...")
print("=" * 60)

# 2. Create zip file with all translations
output_dir = "translations"
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add all translation files from the translations directory
    for root, dirs, files in os.walk(output_dir):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.join('translations', file)
            zipf.write(file_path, arcname)
            print(f"  ✔ Added to zip: {file}")

    # Also add the merged file if it exists
    merged_file = "all_translations_merged.txt"
    if os.path.exists(merged_file):
        zipf.write(merged_file, merged_file)
        print(f"  ✔ Added to zip: {merged_file}")

zip_size = os.path.getsize(zip_filename) / (1024*1024)
print(f"\n✅ Zip file created: '{zip_filename}' ({zip_size:.2f} MB)")

# 3. Mount Google Drive
print("\n" + "=" * 60)
print("Mounting Google Drive...")
drive.mount('/content/drive', force_remount=True)

# 4. Copy zip file to Drive
drive_path = f"/content/drive/MyDrive/{zip_filename}"
print(f"\nUploading to Google Drive...")

import shutil
shutil.copy(zip_filename, drive_path)

print("\n" + "=" * 60)
print("✅ SUCCESS!")
print("=" * 60)
print(f"📦 Zip file uploaded to: MyDrive/{zip_filename}")
print(f"📊 Size: {zip_size:.2f} MB")
print("\nYou can find it in your Google Drive root folder!")
print("=" * 60)

Creating zip file...
  ✔ Added to zip: nepali_ne.json
  ✔ Added to zip: indonesian_id.txt
  ✔ Added to zip: zulu_zu.txt
  ✔ Added to zip: punjabi_pa.json
  ✔ Added to zip: japanese_ja.txt
  ✔ Added to zip: greek_el.txt
  ✔ Added to zip: ewe_ee.txt
  ✔ Added to zip: uzbek_uz.txt
  ✔ Added to zip: icelandic_is.txt
  ✔ Added to zip: macedonian_mk.txt
  ✔ Added to zip: galician_gl.txt
  ✔ Added to zip: norwegian_no.txt
  ✔ Added to zip: aymara_ay.txt
  ✔ Added to zip: myanmar_my.txt
  ✔ Added to zip: bhojpuri_bho.txt
  ✔ Added to zip: malayalam_ml.txt
  ✔ Added to zip: igbo_ig.txt
  ✔ Added to zip: russian_ru.txt
  ✔ Added to zip: chichewa_ny.txt
  ✔ Added to zip: arabic_ar.txt
  ✔ Added to zip: hawaiian_haw.txt
  ✔ Added to zip: haitian_creole_ht.txt
  ✔ Added to zip: serbian_sr.txt
  ✔ Added to zip: kazakh_kk.txt
  ✔ Added to zip: quechua_qu.txt
  ✔ Added to zip: hausa_ha.txt
  ✔ Added to zip: basque_eu.txt
  ✔ Added to zip: vietnamese_vi.json
  ✔ Added to zip: kannada_kn.txt
  ✔ Added t